In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
 #       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install datasets
!pip install torch_snippets

In [ ]:
df=pd.read_csv('/kaggle/input/medicaltranscriptions/mtsamples.csv')
df=df.dropna()
df

In [ ]:
from datasets import Dataset
dataset=Dataset.from_pandas(df)
dataset.train_test_split(train_size=0.95)

In [ ]:
dataset=dataset.remove_columns(['Unnamed: 0','medical_specialty','sample_name','keywords', '__index_level_0__'])

In [ ]:
dataset

In [ ]:
from tqdm import tqdm 
def tokenize(batch):
    tokens=[]
    attention=[]
    for i in tqdm (range(len(dataset))):
        transcription=tokenizer(dataset['transcription'][i],max_length=400,padding=True,truncation=True)
        description=tokenizer(dataset['description'][i],max_length=400,padding=True,truncation=True)
        text=description['input_ids']+transcription['input_ids']
        att_mask=description['attention_mask']+transcription['attention_mask']
        tokens.append(text)
        attention.append(att_mask)
    return tokens,attention
    



In [ ]:
!pip install transformers 

In [ ]:
import torch 
from transformers import AutoTokenizer, GPT2LMHeadModel
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=GPT2LMHeadModel.from_pretrained(model_name).to(device)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokens,attention=tokenize(dataset)
new_dataset=dataset.add_column('input_ids',tokens)
new_dataset=new_dataset.add_column('attention_mask',attention)


In [ ]:
new_dataset

In [ ]:
new_dataset=new_dataset.remove_columns(dataset.column_names)

In [ ]:
new_dataset.set_format('torch')

In [ ]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(new_dataset,batch_size=1,shuffle=True)

In [ ]:
items=next(iter(train_dataloader))
items

In [ ]:
from torch.optim import AdamW
gradient_acc_step=2
optimizer = AdamW(model.parameters(), lr=5e-5,weight_decay=1e-7,eps=1e-6)

In [ ]:
def train_batch(step,batch):
  t=batch['input_ids'][0].unsqueeze(0).to(device)
  att=batch['attention_mask'][0].unsqueeze(0).to(device)
  model.train()
  pred=model(input_ids=t,attention_mask=att,labels=t)
  loss=pred.loss
  loss.backward()
  if (step+1)%gradient_acc_step==0:
      optimizer.step()
      optimizer.zero_grad()
  return loss.item()

In [ ]:
from torch_snippets import *

In [ ]:
n_epoch=10
log=Report(n_epoch)
for epochs in range(n_epoch):
  N = len(train_dataloader)
  for i,data in enumerate(train_dataloader):
    loss=train_batch(i,data)
    log.record(epochs+(i+1)/N,trn_loss=loss,end='\r')
  log.report_avgs(epochs+1)

In [ ]:
tokenizer.pad_token_id=tokenizer.eos_token_id
input_txt = "assisted laparoscopic approach"
with torch.no_grad():
    input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
    output=model.generate(input_ids,max_new_tokens=300,do_sample=True, temperature=0.7, top_p=0.90,top_k=10)
print(tokenizer.decode(output[0]))

In [ ]:
tokenizer.save_pretrained('medical_transcription')
model.save_pretrained('medical_transcription')

In [ ]:
df['description'][3000]